In [1]:
include("src/lisa_store.jl")

using ..Store
using SQLite
using DBInterface
using MurmurHash3
using TextAnalysis
using JSON3
using PooledArrays
using UUIDs
using HDF5
using EasyConfig

db = Graph.DB("lisa.db")
hll = SetCore.HllSet{10}()

# Create an empty HDF5 file. Run it only once, because it will overwrite the file
h5open("lisa_arch.hdf5", "cw") do f
    # The file is now open, but it's empty
end

In [2]:
p = 10
hll = SetCore.HllSet{p}()
Store.book_file(db, "/home/alexmy/JULIA/DEMO/sample/")

In [3]:
uuid = string(uuid4())
df = Graph.set_lock!(db, 
    "/home/alexmy/JULIA/DEMO/sample", 
    "csv", 
    "book_file", 
    "ingest_csv", 
    "waiting", 
    "waiting", uuid; result=true)

for row in eachrow(df)
    assign = Graph.Assignment(row)
    col_uuid = string(uuid4())
    Store.ingest_csv_by_column(db, assign, col_uuid; limit=100, offset=10)
end

Processed column: 2
Processed column: 2


In [4]:
Store.commit(db, "lisa_arch.hdf5", "Alex Mylnikov", "alexmy@lisa-park.com", "commit 2", Config())

save_edge error: HDF5.API.H5Error("Error creating group ///523c6e1b-5203-40ef-974a-e5d5457d8575/edges/0b90b1fee69c77ffa3efe57db7788112ef96dba6/has_column", 1008806316530991107)
save_edge error: HDF5.API.H5Error("Error creating group ///523c6e1b-5203-40ef-974a-e5d5457d8575/edges/6be12bee4edf7c96016907e44bb520be80dc9232/has_column", 1008806316530991108)


In [5]:
hdf = h5open("lisa_arch.hdf5", "r") 

🗂️ HDF5.File: (read-only) lisa_arch.hdf5
├─ 📂 2da19026-e0ca-4729-86af-d183f5aec3ea
│  ├─ 📂 edges
│  │  ├─ 📂 0b90b1fee69c77ffa3efe57db7788112ef96dba6
│  │  │  └─ 📂 has_column
│  │  │     ├─ 🔢 1d29c5326d2292d8717e189a12ca4bd4cbac8b76
│  │  │     │  └─ (5 children)
│  │  │     ├─ 🔢 3f9526f8d331b9519b8632a11b2d344ab7c647b6
│  │  │     │  └─ (5 children)
│  │  │     ├─ 🔢 4b925186fe6753be2ed6908976e44e0f7630d40f
│  │  │     │  └─ (5 children)
│  │  │     ├─ 🔢 4ea0d4a979741fbb57dd4cc9a2c05913a6ee875d
│  │  │     │  └─ (5 children)
│  │  │     ├─ 🔢 5c6ad737758c030a92351e80c15650712fa06108
│  │  │     │  └─ (5 children)
│  │  │     ├─ 🔢 8e5610e809bad1efe796848a86ecb9e28e1709b9
│  │  │     │  └─ (5 children)
│  │  │     ├─ 🔢 d87927e475744f6280feca8fd040dd42d07dba4a
│  │  │     │  └─ (5 children)
│  │  │     └─ 🔢 de6012ead358304ae31a499f29d380878219a256
│  │  │        └─ (5 children)
│  │  └─ 📂 6be12bee4edf7c96016907e44bb520be80dc9232
│  │     └─ 📂 has_column
│  │        ├─ 🔢 0f73a283cd731d2f36ef

In [7]:
# Close the file
close(hdf)

In [8]:
h5open("lisa_arch.hdf5", "r") do h5f
    read(h5f["/2da19026-e0ca-4729-86af-d183f5aec3ea/nodes/0b90b1fee69c77ffa3efe57db7788112ef96dba6"])
end

Dict{String, Any} with 1 entry:
  "[\"csv_file\"]" => [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0,…